# Przygotowanie danych

- Merge danych z różnych stacji
- Wybór kolumn
- Uzupełnianie braków
- Feature engineering wspólny

In [2]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set_theme(style="whitegrid")
DATA_DIR = Path("data")
STATIONS = {
    "Wroclaw": "12424",  # Cel
    "Legnica": "12415",  # Zachód
    "Opole": "12530",  # Wschód
    "Poznan": "12330",  # Północ
    "Klodzko": "12520",  # Południe
}
MAIN_STATION = "Wroclaw"

In [3]:
START_DATE="2018-01-01" # Z dużymi brakami w deszczu, ale pozostałe kolumny w większości są
# START_DATE="2022-05-26" # Pełne dane deszczu

## Dane do prognozy temperatury

- Uzupełnianie ewentualnych braków
- Łączenie danych z różnych stacji
- Podstawowy feature engineering

In [4]:
COLS_MAIN = ['temp', 'rhum', 'prcp', 'snwd', 'wdir', 'wspd', 'wpgt', 'pres', 'cldc', 'coco']
COLS_NEIGHBOR = ['temp', 'pres', 'prcp', 'wspd', 'rhum', 'wdir']
def load_data():
    dfs = []
    for name, station_id in STATIONS.items():
        file_path = DATA_DIR / f"{name.lower()}.csv"
        df = pd.read_csv(file_path, parse_dates=['time'], index_col='time')
        df = df.loc[START_DATE:]

        # Select columns
        target_cols = COLS_MAIN if name == MAIN_STATION else COLS_NEIGHBOR
        available = [c for c in target_cols if c in df.columns]
        df = df[available]

        # Rename columns for neighbors
        if name != MAIN_STATION:
            df.columns = [f"{col}_{name}" for col in df.columns]
        
        dfs.append(df)

    return dfs

In [5]:
dfs = load_data()

In [6]:
def index_gap_report(df, freq="h"):
    idx = df.index.sort_values()

    expected = pd.date_range(
        start=idx.min(),
        end=idx.max(),
        freq=freq
    )

    missing = expected.difference(idx)

    # długości dziur
    diffs = idx.to_series().diff()
    step = pd.to_timedelta(1, unit=freq)

    gap_sizes = diffs[diffs > step]

    if len(gap_sizes) == 0:
        max_gap = pd.Timedelta(0)
    else:
        max_gap = gap_sizes.max()

    print(f"  missing timestamps: {len(missing)}")
    print(f"  longest gap: {max_gap}")

    return missing, gap_sizes

In [7]:
for df in dfs:
  index_gap_report(df)

  missing timestamps: 2
  longest gap: 0 days 03:00:00
  missing timestamps: 16
  longest gap: 0 days 03:00:00
  missing timestamps: 8
  longest gap: 0 days 03:00:00
  missing timestamps: 3
  longest gap: 0 days 03:00:00
  missing timestamps: 8
  longest gap: 0 days 03:00:00


In [8]:
def enforce_continuous_index(df, freq="h"):
    full_idx = pd.date_range(
        start=df.index.min(),
        end=df.index.max(),
        freq=freq
    )
    return df.reindex(full_idx)


In [9]:
dfs_new = []
for df in dfs:
  dfs_new.append(enforce_continuous_index(df))
dfs=dfs_new
for df in dfs:
  index_gap_report(df)

  missing timestamps: 0
  longest gap: 0 days 00:00:00
  missing timestamps: 0
  longest gap: 0 days 00:00:00
  missing timestamps: 0
  longest gap: 0 days 00:00:00
  missing timestamps: 0
  longest gap: 0 days 00:00:00
  missing timestamps: 0
  longest gap: 0 days 00:00:00


In [10]:
for df in dfs:
    print(df.isna().mean() * 100)

temp     0.002853
rhum     0.004279
prcp    55.700735
snwd    99.736110
wdir     0.029955
wspd     0.002853
wpgt     7.233436
pres     0.004279
cldc     3.126738
coco     4.484702
dtype: float64
temp_Legnica     0.022823
pres_Legnica     0.022823
prcp_Legnica    55.700735
wspd_Legnica     0.155481
rhum_Legnica     0.022823
wdir_Legnica     0.022823
dtype: float64
temp_Opole     0.011411
pres_Opole     0.011411
prcp_Opole    55.700735
wspd_Opole     0.128379
rhum_Opole     0.011411
wdir_Opole     0.011411
dtype: float64
temp_Poznan     0.004279
pres_Poznan     0.005706
prcp_Poznan    55.700735
wspd_Poznan     0.004279
rhum_Poznan     0.044219
wdir_Poznan     0.005706
dtype: float64
temp_Klodzko     0.011411
pres_Klodzko     0.011411
prcp_Klodzko    55.700735
wspd_Klodzko     0.232508
rhum_Klodzko     0.011411
wdir_Klodzko     0.159760
dtype: float64


Zasady uzupełniania:

- **temp** - jak godzinę temu
- **rhum** - jak godzinę temu
- **prcp** - 0
- **snwd** - 0
- **wdir** - jak godzinę temu
- **wspd** - jak godzinę temu
- **wpgt** - jak **wspd**
- **pres** - jak godzinę temu
- **tsun** - usuń całą kolumnę (zrobione wcześniej)
- **cldc** - jak godzinę temu
- **coco** - jak godzinę temu, a braki na początku danych 0

In [11]:
def fill_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    # reguły bazowe (bez sufiksów)
    ffill_cols = ['temp', 'rhum', 'wdir', 'wspd', 'pres', 'cldc', 'coco']
    zero_cols = ['prcp', 'snwd', 'coco']

    # 1. zerowanie opadu i śniegu
    for base in zero_cols:
        cols = [c for c in df.columns if c == base or c.startswith(f"{base}_")]
        df[cols] = df[cols].fillna(0)

    # 2. forward fill zmiennych wolnozmiennych
    for base in ffill_cols:
        cols = [c for c in df.columns if c == base or c.startswith(f"{base}_")]
        df[cols] = df[cols].ffill()

    # 3. wpgt = jak wspd
    # stacja główna
    if 'wpgt' in df.columns and 'wspd' in df.columns:
        df['wpgt'] = df['wpgt'].fillna(df['wspd'])

    return df


In [12]:
dfs_new = []
for df in dfs:
  dfs_new.append(fill_missing_values(df))
dfs=dfs_new
for df in dfs:
    print(df.isna().mean() * 100)

temp    0.0
rhum    0.0
prcp    0.0
snwd    0.0
wdir    0.0
wspd    0.0
wpgt    0.0
pres    0.0
cldc    0.0
coco    0.0
dtype: float64
temp_Legnica    0.0
pres_Legnica    0.0
prcp_Legnica    0.0
wspd_Legnica    0.0
rhum_Legnica    0.0
wdir_Legnica    0.0
dtype: float64
temp_Opole    0.0
pres_Opole    0.0
prcp_Opole    0.0
wspd_Opole    0.0
rhum_Opole    0.0
wdir_Opole    0.0
dtype: float64
temp_Poznan    0.0
pres_Poznan    0.0
prcp_Poznan    0.0
wspd_Poznan    0.0
rhum_Poznan    0.0
wdir_Poznan    0.0
dtype: float64
temp_Klodzko    0.0
pres_Klodzko    0.0
prcp_Klodzko    0.0
wspd_Klodzko    0.0
rhum_Klodzko    0.0
wdir_Klodzko    0.0
dtype: float64


In [13]:
# Merge all stations on time index
df_combined = pd.concat(dfs, axis=1).sort_index()
df_combined.dropna()
print(df_combined.isna().mean() * 100)

temp            0.0
rhum            0.0
prcp            0.0
snwd            0.0
wdir            0.0
wspd            0.0
wpgt            0.0
pres            0.0
cldc            0.0
coco            0.0
temp_Legnica    0.0
pres_Legnica    0.0
prcp_Legnica    0.0
wspd_Legnica    0.0
rhum_Legnica    0.0
wdir_Legnica    0.0
temp_Opole      0.0
pres_Opole      0.0
prcp_Opole      0.0
wspd_Opole      0.0
rhum_Opole      0.0
wdir_Opole      0.0
temp_Poznan     0.0
pres_Poznan     0.0
prcp_Poznan     0.0
wspd_Poznan     0.0
rhum_Poznan     0.0
wdir_Poznan     0.0
temp_Klodzko    0.0
pres_Klodzko    0.0
prcp_Klodzko    0.0
wspd_Klodzko    0.0
rhum_Klodzko    0.0
wdir_Klodzko    0.0
dtype: float64


In [14]:
index_gap_report(df_combined)

  missing timestamps: 0
  longest gap: 0 days 00:00:00


(DatetimeIndex([], dtype='datetime64[ns]', freq='h'),
 Series([], Freq: h, dtype: timedelta64[ns]))

In [15]:
def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    idx = df.index
    df['year'] = idx.year
    df['month'] = idx.month
    df['day'] = idx.day
    df['day_of_year'] = idx.dayofyear

    # --- Time cyclical encoding ---
    df['hour_sin'] = np.sin(2 * np.pi * idx.hour / 24)
    df['hour_cos'] = np.cos(2 * np.pi * idx.hour / 24)
    df['day_year_sin'] = np.sin(2 * np.pi * idx.dayofyear / 365)
    df['day_year_cos'] = np.cos(2 * np.pi * idx.dayofyear / 365)
    df['month'] = idx.month

    # --- Spatial gradients (neighbors vs Wroclaw) ---
    for city in STATIONS:
        if city == MAIN_STATION:
            continue

        t_col = f'temp_{city}'
        p_col = f'pres_{city}'

        if t_col in df.columns and 'temp' in df.columns:
            df[f'grad_temp_{city}'] = df[t_col] - df['temp']

        if p_col in df.columns and 'pres' in df.columns:
            df[f'grad_pres_{city}'] = df[p_col] - df['pres']

    return df


In [16]:
df = feature_engineering(df_combined)
df.head()

,temp,rhum,prcp,snwd,wdir,wspd,wpgt,pres,cldc,coco,...,wdir_sin,wdir_cos,grad_temp_Legnica,grad_pres_Legnica,grad_temp_Opole,grad_pres_Opole,grad_temp_Poznan,grad_pres_Poznan,grad_temp_Klodzko,grad_pres_Klodzko
2018-01-01 00:00:00,10.1,72.0,0.0,0.0,220.0,18.0,18.0,1004.3,7.0,0.0,...,-0.642788,-0.766044,-0.4,-0.7,-2.7,2.7,-0.8,-3.2,-3.1,2.6
2018-01-01 01:00:00,10.1,71.0,0.0,0.0,220.0,18.0,18.0,1004.1,7.0,0.0,...,-0.642788,-0.766044,-0.4,-0.9,-3.4,2.3,-0.8,-3.3,-3.5,2.8
2018-01-01 02:00:00,9.8,71.0,0.0,0.0,220.0,14.4,14.4,1003.9,7.0,0.0,...,-0.642788,-0.766044,-0.1,-0.8,-3.5,2.2,-0.8,-3.3,-2.5,2.3
2018-01-01 03:00:00,10.1,69.0,0.0,0.0,230.0,21.6,21.6,1003.6,7.0,0.0,...,-0.766044,-0.642788,-1.2,-0.9,-3.8,1.9,-1.1,-3.3,-3.8,2.4
2018-01-01 04:00:00,10.3,68.0,0.0,0.0,220.0,21.6,21.6,1003.1,7.0,0.0,...,-0.642788,-0.766044,-1.3,-1.1,-3.9,2.2,-1.3,-3.3,-4.1,3.0


In [17]:
df.to_csv(DATA_DIR / "combined.csv", index_label="time")

## Targety z podziałem na dzień i noc do prognozy temperatury

In [18]:
def get_day_night_temp_aggregations(df_raw):
    temp = df_raw[['temp']].copy()
    # Definicja okresów
    mask_day = (temp.index.hour >= 6) & (temp.index.hour <= 17)
    mask_night = (temp.index.hour >= 18) | (temp.index.hour <= 5)

    # 1. Target Dnia (Średnia 06:00 - 17:00 przypisana do daty)
    temp_day = temp.loc[mask_day, 'temp'].resample('D').mean()
    temp_day.name = 'temp_day'

    # 2. Target Nocy (Średnia 18:00 - 05:00 przypisana do daty wieczora)
    # Przesuwamy -6h, żeby noc 1/2 stycznia liczyła się do 1 stycznia
    temp_night_shifted = temp.loc[mask_night].copy()
    temp_night_shifted.index = temp_night_shifted.index - pd.Timedelta(hours=6)
    temp_night = temp_night_shifted['temp'].resample('D').mean()
    temp_night.name = 'Target_Night'

    return temp_day, temp_night

def build_and_save_day_night_targets(df_raw, output_path):
    temp_day, temp_night = get_day_night_temp_aggregations(df_raw)

    # Agregacja dla daty D zawiera dzień D i noc zaczynającą się w D a kończącą w D+1
    # I jest targetem dla danych z datą D-1
    temp_day_target = temp_day.copy()
    temp_day_target.index = temp_day_target.index - pd.Timedelta(days=1)
    temp_day_target.name = 'target_temp_day'

    temp_night_target = temp_night.copy()
    temp_night_target.index = temp_night_target.index - pd.Timedelta(days=1)
    temp_night_target.name = 'target_temp_night'

    targets = pd.concat(
        [temp_day_target, temp_night_target],
        axis=1
    ).sort_index()

    # Usunięcie dni bez pełnych targetów
    targets = targets.dropna()
    targets.to_csv(output_path, index_label='time')

    return targets

In [19]:
targets = build_and_save_day_night_targets(
    df,
    DATA_DIR / 'targets_day_night_temp.csv'
)